In [94]:
#Limpieza
# Limpieza de entorno
%reset -f

In [95]:
# ------------------------------------------------------------
# 1) Librerías y directorio de trabajo
# ------------------------------------------------------------
import os
import pandas as pd
import numpy as np

path = '/Users/usuario/Documents/UBA/2025/Trimestre 3/Taller de programación/TPs/Exposiciones/Exposición 1/Base2024'
os.chdir(path)
print("WD:", os.getcwd())

WD: /Users/usuario/Documents/UBA/2025/Trimestre 3/Taller de programación/TPs/Exposiciones/Exposición 1/Base2024


In [96]:
# ------------------------------------------------------------
# 2) Carga del CSV (separador ';', comillas, BOM)
# ------------------------------------------------------------
fname = 'Latinobarometro_2024_Csv_esp_v20250817.csv'
try:
    df = pd.read_csv(fname, sep=';', quotechar='"', encoding='utf-8-sig', low_memory=False)
except UnicodeDecodeError:
    df = pd.read_csv(fname, sep=';', quotechar='"', encoding='latin1', low_memory=False)

print("Dimensiones iniciales:", df.shape)

Dimensiones iniciales: (19214, 332)


In [97]:
# ------------------------------------------------------------
# 3) Normalización de nombres de columnas
# ------------------------------------------------------------
df.columns = df.columns.str.lower().str.strip().str.replace('.', '_', regex=False)


In [98]:
# ------------------------------------------------------------
# 4) Subconjunto de variables de interés
# (se mantiene tu lista original)
# ------------------------------------------------------------
vars_interes = [
    'idenpa','sexo','edad',
    'p1st','p2st','p6stgbs','p7stgbs','p8st','p10stgbs','p11stgbs',
    'p12stgbs_a','p12stgbs_b','p15stgbs','p16st','p17st','p19st',
    'p23stm_1','p23stm_2','p23stm_3','p27st',
    'p33st_a','p33st_b','p33n_c','p33st_d','p33n_e','p34st',
    'p3ncv','p7ncv_a','p7ncv_b','p7ncv_c','p7ncv_d','p7ncv_e','p7ncv_f','p7ncv_g',
    'p49st_a','p49st_b','p49st_c','p54n',
    's2','s5','s7','s11','s17','s18_a'
]
df = df[[c for c in vars_interes if c in df.columns]].copy()

print("Dimensiones tras selección:", df.shape)
print("Primeras columnas:", df.columns.tolist()[:12])


Dimensiones tras selección: (19214, 44)
Primeras columnas: ['idenpa', 'sexo', 'edad', 'p1st', 'p2st', 'p6stgbs', 'p7stgbs', 'p8st', 'p10stgbs', 'p11stgbs', 'p12stgbs_a', 'p12stgbs_b']


In [99]:
# ------------------------------------------------------------
# 5) Reemplazo de códigos de no respuesta
# ------------------------------------------------------------
missing_codes = [8, 9, 97, 98, 99]
df.replace(missing_codes, np.nan, inplace=True)

# Mantener 0 SOLO en escalas 0–10 indicadas
cols_escalas = ['p16st','p19st','p54n']
for c in df.columns:
    if c not in cols_escalas:
        df[c] = df[c].replace({0: np.nan})

In [100]:
# ------------------------------------------------------------
# 6) Variables derivadas (sin duplicar nombres que renombraremos después)
# ------------------------------------------------------------

df['sexo_lbl'] = df['sexo'].map({1: 'Hombre', 2: 'Mujer'})

# OJO: no crear 'satisfaccion_vida' ni 'satisfaccion_dem' aquí,
# porque luego renombramos p1st -> 'satisfaccion_vida' y p12stgbs_a -> 'satisfaccion_democracia'.

# confianza_media: solo si existen columnas p14stgbs*
cols_conf = df.filter(like='p14stgbs').columns
if len(cols_conf) > 0:
    df['confianza_media'] = df[cols_conf].mean(axis=1)
# si no hay columnas, no crear 'confianza_media' (evita 100% NA)

# Índice de actitudes ambientales (estos ítems sí existen)
df['actitudes_ambientales'] = df[['p7ncv_a','p7ncv_b','p7ncv_c','p7ncv_d','p7ncv_e','p7ncv_f','p7ncv_g']].mean(axis=1)


In [101]:
# ------------------------------------------------------------
# 7) Control de calidad rápido
# ------------------------------------------------------------
print("Dimensiones con derivadas:", df.shape)
print("\n% de NA (top 10):")
print((df.isna().mean().sort_values(ascending=False).head(10)*100).round(1))


Dimensiones con derivadas: (19214, 46)

% de NA (top 10):
p23stm_1    32.1
p23stm_3    29.2
p23stm_2    28.9
p54n        21.4
p16st       16.5
p19st       15.9
s11          5.8
edad         0.0
p7ncv_b      0.0
p7ncv_c      0.0
dtype: float64


In [102]:
# ============================================================
# 8) TABLA DESCRIPTIVA COMPLETA (todas las variables del df actual)
# ============================================================
from IPython.display import display

def resumen_variable_completo(df_):
    resumen = []
    for c in df_.columns:
        serie = df_[c]
        tipo = str(serie.dtype)
        n_total = len(serie)
        n_na = serie.isna().sum()
        n_valid = n_total - n_na
        prop_na = round((n_na / n_total) * 100, 1)

        if np.issubdtype(serie.dropna().dtype, np.number) if n_valid > 0 else False:
            stats = serie.describe(percentiles=[0.25, 0.5, 0.75])
            resumen.append({
                "Variable": c,
                "Tipo": tipo,
                "N válidos": n_valid,
                "% NA": prop_na,
                "Media": round(stats.get("mean", np.nan), 2),
                "Desv.Est.": round(stats.get("std", np.nan), 2),
                "Mín": stats.get("min", np.nan),
                "Máx": stats.get("max", np.nan),
                "Ejemplo/Valores": np.nan
            })
        else:
            valores = serie.dropna().unique()
            ejemplo = ', '.join(map(str, valores[:5]))
            resumen.append({
                "Variable": c,
                "Tipo": tipo,
                "N válidos": n_valid,
                "% NA": prop_na,
                "Media": np.nan,
                "Desv.Est.": np.nan,
                "Mín": np.nan,
                "Máx": np.nan,
                "Ejemplo/Valores": ejemplo
            })
    return pd.DataFrame(resumen).sort_values("Variable").reset_index(drop=True)

tabla_desc = resumen_variable_completo(df)

# Vista en pantalla (primeras 50 filas)
display(tabla_desc.head(50))

# Info útil de tamaño
print(f"Filas en tabla_desc: {len(tabla_desc)}")

,Variable,Tipo,N válidos,% NA,Media,Desv.Est.,Mín,Máx,Ejemplo/Valores
0,actitudes_ambientales,float64,19214,0.0,1.62e+00,7.60e-01,-5.0,4.00e+00,NaN
1,edad,float64,19210,0.0,4.18e+01,1.68e+01,16.0,9.50e+01,NaN
2,idenpa,float64,19214,0.0,3.55e+02,2.63e+02,32.0,8.62e+02,NaN
3,p10stgbs,float64,19214,0.0,1.71e+00,1.01e+00,-5.0,2.00e+00,NaN
4,p11stgbs,float64,19214,0.0,1.52e+00,1.09e+00,-2.0,3.00e+00,NaN
5,p12stgbs_a,float64,19214,0.0,2.70e+00,1.13e+00,-2.0,4.00e+00,NaN
6,p12stgbs_b,float64,19214,0.0,3.03e+00,9.90e-01,-2.0,4.00e+00,NaN
7,p15stgbs,float64,19214,0.0,1.27e+00,1.41e+00,-5.0,2.00e+00,NaN
8,p16st,float64,16050,16.5,4.89e+00,3.43e+00,-2.0,1.00e+01,NaN
9,p17st,float64,19214,0.0,2.91e+00,1.01e+00,-2.0,4.00e+00,NaN


Filas en tabla_desc: 46


In [103]:
# ============================================================
# 9) TABLAS DE FRECUENCIA (todas las variables)
# ============================================================
from IPython.display import display

limite_categorias = 15

def tabla_frecuencias(df_):
    lista_frec = []
    for c in df_.columns:
        serie = df_[c]
        # Categóricas o numéricas con pocos valores
        if (serie.dtype == 'O') or (serie.nunique(dropna=False) <= limite_categorias):
            frec_abs = serie.value_counts(dropna=False)
            frec_rel = serie.value_counts(normalize=True, dropna=False) * 100
            temp = pd.DataFrame({
                'Variable': c,
                'Valor': frec_abs.index.astype(object),
                'Frecuencia': frec_abs.values,
                '%': frec_rel.round(2).values
            })
            lista_frec.append(temp)
        else:
            temp = pd.DataFrame({
                'Variable': [c],
                'Valor': ['(numérica continua)'],
                'Frecuencia': [len(serie.dropna())],
                '%': [100.0]
            })
            lista_frec.append(temp)
    return pd.concat(lista_frec, ignore_index=True)

tablas_frec = tabla_frecuencias(df)

# Vista en pantalla (primeras 100 filas)
display(tablas_frec.head(100))

# Tip: inspeccionar una variable específica
# display(tablas_frec[tablas_frec['Variable'] == 'satisfaccion_vida'])
print(f"Filas en tablas_frec: {len(tablas_frec)}")

,Variable,Valor,Frecuencia,%
0,idenpa,(numérica continua),19214,100.00
1,sexo,2.0,9940,51.73
2,sexo,1.0,9274,48.27
3,edad,(numérica continua),19210,100.00
4,p1st,2.0,7648,39.80
...,...,...,...,...
95,p23stm_1,7.0,788,4.10
96,p23stm_1,-5.0,386,2.01
97,p23stm_2,NaN,5562,28.95
98,p23stm_2,6.0,2803,14.59


Filas en tablas_frec: 253


In [104]:
# ============================================================
# 10) LIMPIEZA FINAL DE CÓDIGOS NEGATIVOS
# ============================================================
codigos_negativos = [-1, -2, -3, -4, -5, -9, -99]
cols_excluir = ['idenpa', 'sexo', 'edad']

for c in df.columns:
    if c not in cols_excluir:
        df[c] = df[c].replace(codigos_negativos, np.nan)

# Verificación (muestras)
for var in ['p1st', 'p2st', 'p6stgbs', 'p10stgbs', 'p11stgbs', 'p12stgbs_a']:
    if var in df.columns:
        print(f"\n{var}:\n", df[var].value_counts(dropna=False).head(10))





p1st:
 p1st
2.0    7648
1.0    7542
3.0    3096
4.0     781
NaN     147
Name: count, dtype: int64

p2st:
 p2st
2.0    8658
1.0    5401
3.0    4709
NaN     446
Name: count, dtype: int64

p6stgbs:
 p6stgbs
3.0    8673
4.0    4899
5.0    2712
2.0    2499
1.0     334
NaN      97
Name: count, dtype: int64

p10stgbs:
 p10stgbs
2.0    15931
1.0     2912
NaN      371
Name: count, dtype: int64

p11stgbs:
 p11stgbs
1.0    10042
3.0     4756
2.0     3154
NaN     1262
Name: count, dtype: int64

p12stgbs_a:
 p12stgbs_a
3.0    7616
4.0    4750
2.0    4188
1.0    2165
NaN     495
Name: count, dtype: int64


In [105]:
# ============================================================
# 11) RENOMBRAR VARIABLES A NOMBRES FINALES
# ============================================================
# Evitar duplicados de columnas si quedaron
df = df.loc[:, ~df.columns.duplicated(keep='first')].copy()

rename_map = {
    'idenpa': 'pais',
    'sexo': 'sexo',
    'edad': 'edad',
    'p1st': 'satisfaccion_vida',
    'p2st': 'situacion_pais',
    'p6stgbs': 'economia_actual',
    'p7stgbs': 'economia_vs12m',
    'p8st': 'economia_12m_futuro',
    'p10stgbs': 'confianza_interpersonal',
    'p11stgbs': 'preferencia_democracia',
    'p12stgbs_a': 'satisfaccion_democracia',
    'p12stgbs_b': 'satisfaccion_economia',
    'p15stgbs': 'aprueba_presidente',
    'p16st': 'ideologia_izq_der',
    'p17st': 'justicia_distributiva',
    'p19st': 'nivel_democratico_pais',
    'p23stm_1': 'poder_1',
    'p23stm_2': 'poder_2',
    'p23stm_3': 'poder_3',
    'p27st': 'elecciones_limpias',
    'p33st_a': 'inmigrantes_ayudan_economia',
    'p33st_b': 'inmigrantes_compiten_trabajo',
    'p33n_c': 'inmigrantes_aumentan_crimen',
    'p33st_d': 'mercado_unico_desarrollo',
    'p33n_e': 'justicia_no_dura_con_delito',
    'p34st': 'impacto_inmigracion_pais',
    'p3ncv': 'existe_cambio_climatico',
    'p7ncv_a': 'me_importa_medioambiente',
    'p7ncv_b': 'obligacion_proteger_medioambiente',
    'p7ncv_c': 'importa_impacto_otros',
    'p7ncv_d': 'proteger_para_futuras_gen',
    'p7ncv_e': 'dispuesto_ahorrar_electricidad',
    'p7ncv_f': 'dispuesto_cambiar_alimentacion',
    'p7ncv_g': 'derecho_plantas_animales',
    'p49st_a': 'delincuencia_ult12m',
    'p49st_b': 'drogadiccion_ult12m',
    'p49st_c': 'corrupcion_ult12m',
    'p54n': 'corrupcion_0a10',
    's2': 'clase_social',
    's5': 'suficiencia_ingresos',
    's7': 'raza_etnia',
    's11': 'educacion_anios',
    's17': 'fecha_nacimiento',
    's18_a': 'situacion_ocupacional'
}
df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)



In [106]:
# ============================================================
# 12) ETIQUETADO (crea *_lbl ordenadas)
# ============================================================
lab_sexo = {1: 'Hombre', 2: 'Mujer'}
lab_satisf_1a4 = {1:'Muy satisfecho/a',2:'Bastante satisfecho/a',3:'Poco satisfecho/a',4:'Nada satisfecho/a'}
lab_situacion_pais = {1:'Progresando',2:'Estancado',3:'En retroceso'}
lab_econ_1a5 = {1:'Muy buena',2:'Buena',3:'Regular',4:'Mala',5:'Muy mala'}
lab_mejor_igual_peor = {1:'Mucho mejor',2:'Un poco mejor',3:'Igual',4:'Un poco peor',5:'Mucho peor'}
lab_confianza_interpersonal = {1:'Se puede confiar en la mayoría',2:'Nunca se es suficientemente cuidadoso'}
lab_pref_democracia = {1:'Prefiere democracia',2:'Autoritarismo puede ser preferible',3:'Le da lo mismo'}
lab_justicia_distributiva = {1:'Muy justa',2:'Justa',3:'Injusta',4:'Muy injusta'}
lab_elecciones = {1:'Limpias',2:'Fraudulentas'}
lab_acuerdo_1a4 = {1:'Muy de acuerdo',2:'De acuerdo',3:'En desacuerdo',4:'Muy en desacuerdo'}
lab_impacto_inmigracion = {1:'Beneficia',2:'Perjudica',3:'Ni beneficia ni perjudica'}
lab_si_no = {1:'Sí',2:'No'}
lab_clase_social = {1:'Alta',2:'Media alta',3:'Media',4:'Media baja',5:'Baja'}
lab_suf_ingresos = {1:'Muy suficiente',2:'Algo suficiente',3:'Poco suficiente',4:'Nada suficiente'}
lab_raza = {1:'Asiático/a',2:'Negro/a',3:'Indígena',4:'Mestizo/a',5:'Mulato/a',6:'Blanco/a',7:'Otra'}
lab_ocupacion = {1:'Independiente/cuenta propia',2:'Asalariado sector público',3:'Asalariado sector privado',
                 4:'Temporalmente no trabaja',5:'Jubilado/a o pensionado/a',6:'Tareas del hogar',7:'Estudiante'}
lab_ult12m = {1:'Aumentó mucho',2:'Aumentó poco',3:'Igual',4:'Disminuyó poco',5:'Disminuyó mucho'}

def etiquetar(df_, var, mapa, orden=None):
    """Crea var+'_lbl' como categoría ordenada a partir de codificación numérica."""
    if var not in df_.columns:
        return
    s = df_[var]
    # si por algún motivo fuera DataFrame (duplicados), tomar primera col
    if isinstance(s, pd.DataFrame):
        s = s.iloc[:, 0]
    ser_mapped = s.map(mapa)  # <- corrección del typo
    if orden is None:
        orden = list(mapa.keys())
    categorias = [mapa[k] for k in orden if k in mapa]
    df_[var + '_lbl'] = pd.Categorical(ser_mapped, categories=categorias, ordered=True)

# Aplicación de etiquetas
etiquetar(df, 'sexo', lab_sexo, orden=[1,2])

for v in ['satisfaccion_vida','satisfaccion_democracia','satisfaccion_economia']:
    etiquetar(df, v, lab_satisf_1a4, orden=[1,2,3,4])

etiquetar(df, 'situacion_pais', lab_situacion_pais, orden=[1,2,3])
etiquetar(df, 'economia_actual', lab_econ_1a5, orden=[1,2,3,4,5])
etiquetar(df, 'economia_vs12m', lab_mejor_igual_peor, orden=[1,2,3,4,5])
etiquetar(df, 'economia_12m_futuro', lab_mejor_igual_peor, orden=[1,2,3,4,5])

etiquetar(df, 'confianza_interpersonal', lab_confianza_interpersonal, orden=[1,2])
etiquetar(df, 'preferencia_democracia', lab_pref_democracia, orden=[1,2,3])

# Ideología numérica + bandas
if 'ideologia_izq_der' in df.columns:
    df['ideologia_izq_der'] = pd.to_numeric(df['ideologia_izq_der'], errors='coerce')
    df['ideologia_bandas'] = pd.cut(
        df['ideologia_izq_der'],
        bins=[-0.1, 3, 7, 10],
        labels=['Izquierda (0–3)', 'Centro (4–7)', 'Derecha (8–10)'],
        include_lowest=True, ordered=True
    )

etiquetar(df, 'justicia_distributiva', lab_justicia_distributiva, orden=[1,2,3,4])

for v in ['nivel_democratico_pais','corrupcion_0a10']:
    if v in df.columns:
        df[v] = pd.to_numeric(df[v], errors='coerce')

etiquetar(df, 'elecciones_limpias', lab_elecciones, orden=[1,2])

for v in ['inmigrantes_ayudan_economia','inmigrantes_compiten_trabajo',
          'inmigrantes_aumentan_crimen','mercado_unico_desarrollo',
          'justicia_no_dura_con_delito']:
    etiquetar(df, v, lab_acuerdo_1a4, orden=[1,2,3,4])

etiquetar(df, 'impacto_inmigracion_pais', lab_impacto_inmigracion, orden=[1,2,3])

etiquetar(df, 'existe_cambio_climatico', lab_si_no, orden=[1,2])
for v in ['me_importa_medioambiente','obligacion_proteger_medioambiente','importa_impacto_otros',
          'proteger_para_futuras_gen','dispuesto_ahorrar_electricidad',
          'dispuesto_cambiar_alimentacion','derecho_plantas_animales']:
    etiquetar(df, v, lab_acuerdo_1a4, orden=[1,2,3,4])

for v in ['delincuencia_ult12m','drogadiccion_ult12m','corrupcion_ult12m']:
    etiquetar(df, v, lab_ult12m, orden=[1,2,3,4,5])

etiquetar(df, 'clase_social', lab_clase_social, orden=[1,2,3,4,5])
etiquetar(df, 'suficiencia_ingresos', lab_suf_ingresos, orden=[1,2,3,4])
etiquetar(df, 'raza_etnia', lab_raza, orden=[1,2,3,4,5,6,7])
etiquetar(df, 'situacion_ocupacional', lab_ocupacion, orden=[1,2,3,4,5,6,7])


In [107]:
# ============================================================
# 13) ORDEN DE SALIDA Y DATASET LIMPIO (df_limpio)
# ============================================================
cols_prioridad = [
    'pais','sexo','sexo_lbl','edad',
    'satisfaccion_vida','satisfaccion_vida_lbl',
    'satisfaccion_democracia','satisfaccion_democracia_lbl',
    'satisfaccion_economia','satisfaccion_economia_lbl',
    'situacion_pais','situacion_pais_lbl',
    'economia_actual','economia_actual_lbl',
    'economia_vs12m','economia_vs12m_lbl',
    'economia_12m_futuro','economia_12m_futuro_lbl',
    'confianza_interpersonal','confianza_interpersonal_lbl',
    'preferencia_democracia','preferencia_democracia_lbl',
    'ideologia_izq_der','ideologia_bandas',
    'justicia_distributiva','justicia_distributiva_lbl',
    'nivel_democratico_pais','corrupcion_0a10',
    'elecciones_limpias','elecciones_limpias_lbl',
    'inmigrantes_ayudan_economia','inmigrantes_ayudan_economia_lbl',
    'inmigrantes_compiten_trabajo','inmigrantes_compiten_trabajo_lbl',
    'inmigrantes_aumentan_crimen','inmigrantes_aumentan_crimen_lbl',
    'mercado_unico_desarrollo','mercado_unico_desarrollo_lbl',
    'justicia_no_dura_con_delito','justicia_no_dura_con_delito_lbl',
    'impacto_inmigracion_pais','impacto_inmigracion_pais_lbl',
    'existe_cambio_climatico','existe_cambio_climatico_lbl',
    'me_importa_medioambiente','me_importa_medioambiente_lbl',
    'obligacion_proteger_medioambiente','obligacion_proteger_medioambiente_lbl',
    'importa_impacto_otros','importa_impacto_otros_lbl',
    'proteger_para_futuras_gen','proteger_para_futuras_gen_lbl',
    'dispuesto_ahorrar_electricidad','dispuesto_ahorrar_electricidad_lbl',
    'dispuesto_cambiar_alimentacion','dispuesto_cambiar_alimentacion_lbl',
    'derecho_plantas_animales','derecho_plantas_animales_lbl',
    'delincuencia_ult12m','delincuencia_ult12m_lbl',
    'drogadiccion_ult12m','drogadiccion_ult12m_lbl',
    'corrupcion_ult12m','corrupcion_ult12m_lbl',
    'clase_social','clase_social_lbl',
    'suficiencia_ingresos','suficiencia_ingresos_lbl',
    'raza_etnia','raza_etnia_lbl',
    'educacion_anios','situacion_ocupacional','situacion_ocupacional_lbl'
]
df_limpio = df[[c for c in cols_prioridad if c in df.columns]].copy()
print("df_limpio → filas, columnas:", df_limpio.shape)


df_limpio → filas, columnas: (19214, 73)


In [108]:
# ============================================================
# 14) Dataset FINAL: 1 columna por variable (con etiqueta)
#     - Categóricas: usar *_lbl y renombrar al nombre final
#     - Numéricas/continuas: mantener numérico
# ============================================================
import os

# Lista de nombres finales (los del rename_map ya aplicado)
vars_finales = [
    'pais','sexo','edad',
    'satisfaccion_vida','situacion_pais','economia_actual','economia_vs12m','economia_12m_futuro',
    'confianza_interpersonal','preferencia_democracia','satisfaccion_democracia','satisfaccion_economia',
    'aprueba_presidente','ideologia_izq_der','justicia_distributiva','nivel_democratico_pais',
    'poder_1','poder_2','poder_3','elecciones_limpias',
    'inmigrantes_ayudan_economia','inmigrantes_compiten_trabajo','inmigrantes_aumentan_crimen',
    'mercado_unico_desarrollo','justicia_no_dura_con_delito','impacto_inmigracion_pais',
    'existe_cambio_climatico','me_importa_medioambiente','obligacion_proteger_medioambiente',
    'importa_impacto_otros','proteger_para_futuras_gen','dispuesto_ahorrar_electricidad',
    'dispuesto_cambiar_alimentacion','derecho_plantas_animales',
    'delincuencia_ult12m','drogadiccion_ult12m','corrupcion_ult12m','corrupcion_0a10',
    'clase_social','suficiencia_ingresos','raza_etnia','educacion_anios','situacion_ocupacional'
]

# Qué variables trataremos como numéricas “continuas/escala”
numericas_puras = {'edad','ideologia_izq_der','nivel_democratico_pais','corrupcion_0a10','educacion_anios','pais'}

# Construir df_final con 1 columna por concepto
cols_existentes = [v for v in vars_finales if v in df.columns or (v+'_lbl') in df.columns]
df_final = pd.DataFrame(index=df.index)

for v in cols_existentes:
    if v in numericas_puras:
        # mantener numérico si existe; sino, si solo hay *_lbl, lo pasamos a string
        if v in df.columns:
            df_final[v] = pd.to_numeric(df[v], errors='coerce')
        elif (v + '_lbl') in df.columns:
            df_final[v] = df[v + '_lbl'].astype(str)
    else:
        # preferir etiqueta si existe; si no, dejar el código
        if (v + '_lbl') in df.columns:
            df_final[v] = df[v + '_lbl'].astype(str)
        elif v in df.columns:
            df_final[v] = df[v]
        else:
            # si no existe ni base ni label, lo omitimos
            pass

# Seguridad: columnas únicas y sin duplicados
df_final = df_final.loc[:, ~df_final.columns.duplicated(keep='first')].copy()

print("df_final → filas, columnas:", df_final.shape)
print("Primeras columnas finales:", list(df_final.columns)[:12])


df_final → filas, columnas: (19214, 43)
Primeras columnas finales: ['pais', 'sexo', 'edad', 'satisfaccion_vida', 'situacion_pais', 'economia_actual', 'economia_vs12m', 'economia_12m_futuro', 'confianza_interpersonal', 'preferencia_democracia', 'satisfaccion_democracia', 'satisfaccion_economia']


In [109]:
# ============================================================
# 15) DICCIONARIO + FRECUENCIAS CATEGÓRICAS + DESCRIPTIVOS
# ============================================================
# DICCIONARIO
dicc_rows = []
for col in df_limpio.columns:
    if col.endswith('_lbl'):
        base = col[:-4]
        if hasattr(df_limpio[col], 'cat') and df_limpio[col].dtype.name == 'category':
            cats = df_limpio[col].cat.categories.tolist()
        else:
            cats = sorted(pd.Series(df_limpio[col].dropna().unique(), dtype=object))
        dicc_rows.append({
            'variable_codigo': base,
            'variable_etiqueta': col,
            'categorias': ', '.join(map(str, cats))
        })
    else:
        if (col + '_lbl') not in df_limpio.columns:
            dicc_rows.append({
                'variable_codigo': col,
                'variable_etiqueta': '',
                'categorias': ''
            })
diccionario = pd.DataFrame(dicc_rows).drop_duplicates(subset=['variable_codigo']).sort_values('variable_codigo')

# FRECUENCIAS CATEGÓRICAS
def frecuencias_categoricas(df_):
    tablas = []
    lbl_cols = [c for c in df_.columns if c.endswith('_lbl')]
    total = len(df_)
    for c in lbl_cols:
        s = df_[c]
        val_count = s.value_counts(dropna=False)
        pct_total = (val_count / total * 100).round(2)

        # % sobre válidos
        valid_n = s.notna().sum()
        pct_valid = (s.value_counts(dropna=True) / (valid_n if valid_n > 0 else 1) * 100).round(2)

        t = pd.DataFrame({
            'variable': c[:-4],
            'categoria': val_count.index.astype(object),
            'frecuencia': val_count.values,
            '%_sobre_total': pct_total.values
        })
        # map % válidos (NaN para la categoría NaN)
        t['%_sobre_validos'] = t['categoria'].map(pct_valid)
        tablas.append(t)
    return pd.concat(tablas, ignore_index=True) if tablas else pd.DataFrame()

freq_cat = frecuencias_categoricas(df_limpio)

# DESCRIPTIVOS NUMÉRICOS
def descriptivos_numericos(df_, vars_num=None):
    if vars_num is None:
        vars_num = [c for c in df_.columns if (not c.endswith('_lbl')) and pd.api.types.is_numeric_dtype(df_[c])]
    rows = []
    n_total = len(df_)
    for v in vars_num:
        s = pd.to_numeric(df_[v], errors='coerce')
        n_valid = s.notna().sum()
        rows.append({
            'variable': v,
            'N_validos': n_valid,
            '%NA': round((1 - n_valid/n_total)*100, 1),
            'media': round(s.mean(), 2),
            'desv_est': round(s.std(), 2),
            'min': s.min(),
            'max': s.max()
        })
    return pd.DataFrame(rows).sort_values('variable')

vars_num = [v for v in ['edad','ideologia_izq_der','nivel_democratico_pais','corrupcion_0a10','educacion_anios'] if v in df_limpio.columns]
desc_num = descriptivos_numericos(df_limpio, vars_num=vars_num)

In [110]:
# ============================================================
# 16) EXPORTACIÓN — FINALES (CSV + Diccionario)
# ============================================================
import os

# a) Base final (una columna por variable, con etiquetas)
out_csv = os.path.abspath("Latinobarometro2024_final.csv")
df_final.to_csv(out_csv, index=False, encoding="utf-8")
print("CSV creado en:", out_csv)

# b) Diccionario de variables finales (categóricas vs numéricas)
dicc_rows = []
for col in df_final.columns:
    s = df_final[col]
    if s.dtype == 'O' or s.nunique(dropna=True) <= 20:
        cats = sorted(s.dropna().unique())
        dicc_rows.append({
            'variable': col,
            'tipo': 'categórica',
            'categorías': ', '.join(map(str, cats[:50])) + ('...' if len(cats) > 50 else '')
        })
    else:
        dicc_rows.append({
            'variable': col,
            'tipo': 'numérica',
            'categorías': ''
        })

diccionario_final = pd.DataFrame(dicc_rows)

# Exportar diccionario (Excel y CSV)
out_xlsx = os.path.abspath("Latinobarometro2024_diccionario_final.xlsx")
out_dcsv = os.path.abspath("Latinobarometro2024_diccionario_final.csv")
diccionario_final.to_excel(out_xlsx, index=False)
diccionario_final.to_csv(out_dcsv, index=False, encoding="utf-8")
print("Diccionario creado en:", out_xlsx)
print("Diccionario (CSV) creado en:", out_dcsv)


CSV creado en: /Users/usuario/Documents/UBA/2025/Trimestre 3/Taller de programación/TPs/Exposiciones/Exposición 1/Base2024/Latinobarometro2024_final.csv
Diccionario creado en: /Users/usuario/Documents/UBA/2025/Trimestre 3/Taller de programación/TPs/Exposiciones/Exposición 1/Base2024/Latinobarometro2024_diccionario_final.xlsx
Diccionario (CSV) creado en: /Users/usuario/Documents/UBA/2025/Trimestre 3/Taller de programación/TPs/Exposiciones/Exposición 1/Base2024/Latinobarometro2024_diccionario_final.csv
